Проверка данных по покупкам 
----
Проверка соответсвие данных с админок GooglePlay и Appstore с данными в Kibana и Amplitude

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Google
----

Чтение нужной страницы в xlsx файле

In [2]:
G2006 = pd.read_excel(open('Jun 2020.xlsx', 'rb'),
              sheet_name='G2006') 
G2006.head(5)

,Description,Transaction Date,Transaction Time,Tax Type,Transaction Type,Refund Type,Product Title,Sku Id,Buyer Country,Buyer Currency,Amount (Buyer Currency),Currency Conversion Rate,Merchant Currency,Amount (Merchant Currency),267426.0499999931,Received
0,GPA.3330-3349-0825-10817,"Jun 1, 2020",12:01:01 AM PDT,NaN,Charge,NaN,Premium Kit II (Tacticool - 5v5 shooter),com.panzerdog.tacticool.premium2,MX,MXN,205.0,0.040772,EUR,8.36,363458.80,Sales
1,GPA.3330-3349-0825-10817,"Jun 1, 2020",12:01:01 AM PDT,NaN,Google fee,NaN,Premium Kit II (Tacticool - 5v5 shooter),com.panzerdog.tacticool.premium2,MX,MXN,-61.5,0.040772,EUR,-2.51,-93502.85,Google fee
2,GPA.3387-8656-4018-02443,"Jun 1, 2020",12:01:11 AM PDT,NaN,Charge,NaN,CLUB PASS (Tacticool - 5v5 shooter),com.panzerdog.tacticool.premium_subscription,VN,VND,119000.0,0.000039,EUR,4.59,-2529.90,Local tax
3,GPA.3387-8656-4018-02443,"Jun 1, 2020",12:01:11 AM PDT,NaN,Google fee,NaN,CLUB PASS (Tacticool - 5v5 shooter),com.panzerdog.tacticool.premium_subscription,VN,VND,-35700.0,0.000039,EUR,-1.38,NaN,NaN
4,GPA.3325-5255-2963-23887,"Jun 1, 2020",12:02:21 AM PDT,NaN,Charge,NaN,PAS KELAB (Tacticool - permainan tembakan 5lwn5),com.panzerdog.tacticool.premium_subscription,ID,IDR,69000.0,0.000061,EUR,4.24,NaN,NaN


Берутся только те данных у которых 'Transaction Type' == 'Charge', остальное не интересно  

In [18]:
google_purchase = G2006[G2006['Transaction Type'] == 'Charge']
google_purchase = google_purchase.groupby(['Sku Id']).count()
google_purchase.head(5)
google_purchase = google_purchase[['Description']]
google_purchase.columns = ['Count']
google_purchase.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55 entries, com.pancerdog.tacticool.tier.32 to com.panzerdog.tacticool.weapon.sr.pro
Data columns (total 2 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   Count                       55 non-null     int64
 1   Amount (Merchant Currency)  55 non-null     int64
dtypes: int64(2)
memory usage: 1.3+ KB


Берем данные из Amplitude

In [4]:
google_amplitude = pd.read_csv('Amplitude Amdroid 2006.csv', delimiter = ',"', header = None, names=['Sku Id', 'AmplitudeCount'], engine='python')
google_amplitude = google_amplitude.replace({'"':''}, regex=True)
google_amplitude = google_amplitude.replace({'\t':''}, regex=True)
google_amplitude.drop(google_amplitude.index[:1], inplace=True)
google_amplitude.sort_values(by = 'Sku Id')
google_amplitude.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 1 to 55
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Sku Id          55 non-null     object
 1   AmplitudeCount  55 non-null     object
dtypes: object(2)
memory usage: 1.3+ KB


Берем данные из Kibana

In [5]:
google_kibana = pd.read_csv('Kibana Andriod 2006.csv', delimiter = ',', header = None, names=['Sku Id', 'KibanaCount'], engine='python')
google_kibana.drop(google_kibana.index[:1], inplace=True)
google_kibana.sort_values(by = 'Sku Id')
google_kibana.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 1 to 55
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Sku Id       55 non-null     object
 1   KibanaCount  55 non-null     object
dtypes: object(2)
memory usage: 1.3+ KB


Напишим функцию для окраски строк, в которых модуль целевого признака больше некоторого значения

In [6]:
def highlight_greaterthan(s, threshold, column):
    is_max = pd.Series(data=False, index=s.index)
    is_max[column] = abs(s.loc[column].astype('int64')) >= threshold
    return ['background-color: yellow' if is_max.any() else '' for v in is_max]

Смержим по full-outer join наши DataFrame и окрасим строки с большой разницой в покупках

In [7]:
mergedStuff = pd.merge(google_purchase, google_amplitude, on=['Sku Id'], how = 'outer')
mergedStuff.fillna(0)
mergedStuff['AmplitudeCount'] = mergedStuff['AmplitudeCount'].astype('int64')

mergedStuff['AmplitudeDiff'] = round((mergedStuff['AmplitudeCount'] - mergedStuff['Count'])*100 / mergedStuff['Count'])

mergedStuff = pd.merge(mergedStuff, google_kibana, on=['Sku Id'], how = 'left')
mergedStuff['KibanaCount'] = mergedStuff['KibanaCount'].astype('int64')

mergedStuff['KibanaDiff'] = round((mergedStuff['KibanaCount'] - mergedStuff['Count'])*100 / mergedStuff['Count'])

mergedStuff = mergedStuff.style.apply(highlight_greaterthan, threshold=10.0, column=['AmplitudeDiff', 'KibanaDiff'], axis=1)
mergedStuff

,Sku Id,Count,AmplitudeCount,AmplitudeDiff,KibanaCount,KibanaDiff
0,com.pancerdog.tacticool.tier.32,393,394,0.000000,373,-5.000000
1,com.panzerdog.tacticool.cbp.10,1296,1276,-2.000000,1241,-4.000000
2,com.panzerdog.tacticool.cbp.15,1263,1258,-0.000000,1241,-2.000000
3,com.panzerdog.tacticool.cbp.5,4234,4232,-0.000000,4144,-2.000000
4,com.panzerdog.tacticool.hero.common.acc,265,265,0.000000,255,-4.000000
5,com.panzerdog.tacticool.hero.common.ammo,1219,1229,1.000000,1370,12.000000
6,com.panzerdog.tacticool.hero.rare,203,203,0.000000,194,-4.000000
7,com.panzerdog.tacticool.hero.uncommon.accuracy,49,48,-2.000000,47,-4.000000
8,com.panzerdog.tacticool.hero.uncommon.ammo,47,47,0.000000,46,-2.000000
9,com.panzerdog.tacticool.hero.uncommon.firerate,25,25,0.000000,23,-8.000000


Выгрузим в xlsx

In [8]:
mergedStuff.to_excel("Purchase Andriod.xlsx") 

AppStore
---

Бреем нужную страничку из xlsx и проделываем те же махинации, что и с google

In [9]:
A2006 = pd.read_excel(open('Jun 2020.xlsx', 'rb'),
              sheet_name='A2006') 
A2006

,SKU,Title,Product Type Identifier,Units,Developer Proceeds,Total proceeds,Customer Currency,Country Code,Currency of Proceeds,Apple Identifier,...,Unnamed: 15,National currencies only,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,EUR,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,com.panzerdog.tacticool.weapon.mg.mk4,Weapon MG MK IV,IA1,11,41.16,452.76,CNY,CN,CNY,1441423170,...,Currency,Total Charged,Developer Proceeds,Apple fee (30%),VAT paid,Exchange rate (30.06.2020),Gross Revenue,VAT paid (EU only),Apple fee (30%),Net Revenue
1,com.panzerdog.tacticool.tier5,com.panzerdog.tacticool.tier5,IA1,50,3.18,159.00,EUR,NL,EUR,1496612502,...,AED,13473.9,8984.75,3850.61,638.553,4.13,3107.83,154.613,932.35,2175.48
2,com.panzerdog.tacticool.premium1,Premium Kit I,IA1,3,7.33,21.99,SAR,SA,SAR,1391775544,...,USD,413.86,242.17,103.787,67.9029,1.1198,308.945,60.6384,92.6836,216.262
3,com.panzerdog.tacticool.kit.starter.fake,Starter Kit Discount,IA1,1,2.80,2.80,USD,AR,USD,1440364321,...,USD,275.13,161.52,69.2229,44.3871,1.1198,206.057,39.6385,61.8172,144.24
4,com.panzerdog.tacticool.tier.19,com.panzerdog.tacticool.tier.19,IA1,17,13.30,226.10,USD,US,USD,1508142941,...,USD,1420.28,998.4,427.886,-6.00571,1.1198,1273.7,-5.3632,382.109,891.588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3476,com.panzerdog.tacticool.clubpass,Regular Club Pass,IAY,1,3.15,3.15,USD,PA,USD,1441905198,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3477,com.panzerdog.tacticool.hero.uncommon.accuracy,Upgrade Points for Hero Uncommon Accuracy,IA1,1,16.44,16.44,MYR,MY,MYR,1441093211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3478,com.panzerdog.tacticool.hero.rare,Upgrade Points for Hero Rare,IA1,2,68.60,137.20,HKD,HK,HKD,1441094184,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3479,com.panzerdog.tacticool.specialsfake,Special Kit Discount,IA1,1,41.93,41.93,PEN,PE,PEN,1441299067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
appstore_purchase = A2006[A2006['Units'] > 0]
appstore_purchase = appstore_purchase.groupby(['SKU']).sum()
appstore_purchase = appstore_purchase[['Units']]
appstore_purchase.columns = ['Count']
appstore_purchase.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55 entries, com.pancerdog.tacticool.tier.32 to com.panzerdog.tacticool.weapon.sr.pro
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Count   55 non-null     int64
dtypes: int64(1)
memory usage: 880.0+ bytes


In [11]:
appstore_amplitude = pd.read_csv('Amplitude IPhone 2006.csv', delimiter = ',"', header = None, names=['SKU', 'AmplitudeCount'], engine='python')
appstore_amplitude = appstore_amplitude.replace({'"':''}, regex=True)
appstore_amplitude = appstore_amplitude.replace({'\t':''}, regex=True)
appstore_amplitude = appstore_amplitude.replace({'\t':''}, regex=True)
appstore_amplitude.drop(appstore_amplitude.index[:1], inplace=True)
appstore_amplitude.sort_values(by = 'SKU')
appstore_amplitude.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 1 to 55
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   SKU             55 non-null     object
 1   AmplitudeCount  55 non-null     object
dtypes: object(2)
memory usage: 1.3+ KB


In [12]:
appstore_kibana = pd.read_csv('Kibana IPhone 2006.csv', delimiter = ',', header = None, names=['SKU', 'KibanaCount'], engine='python')
appstore_kibana.drop(appstore_kibana.index[:1], inplace=True)
appstore_kibana.sort_values(by = 'SKU')
appstore_kibana.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 1 to 55
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   SKU          55 non-null     object
 1   KibanaCount  55 non-null     object
dtypes: object(2)
memory usage: 1.3+ KB


In [13]:
mergedStuff = pd.merge(appstore_purchase, appstore_amplitude, on=['SKU'], how = 'outer')
mergedStuff.fillna(0)
mergedStuff['AmplitudeCount'] = mergedStuff['AmplitudeCount'].astype('int64')
mergedStuff['AmplitudeDiff'] = round((mergedStuff['AmplitudeCount'] - mergedStuff['Count'])*100 / mergedStuff['Count'])

mergedStuff = pd.merge(mergedStuff, appstore_kibana, on=['SKU'], how = 'left')
mergedStuff['KibanaCount'] = mergedStuff['KibanaCount'].astype('int64')
mergedStuff['KibanaDiff'] = round((mergedStuff['KibanaCount'] - mergedStuff['Count'])*100 / mergedStuff['Count'])

mergedStuff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 0 to 54
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SKU             55 non-null     object 
 1   Count           55 non-null     int64  
 2   AmplitudeCount  55 non-null     int64  
 3   AmplitudeDiff   55 non-null     float64
 4   KibanaCount     55 non-null     int64  
 5   KibanaDiff      55 non-null     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 3.0+ KB


In [14]:
mergedStuff = mergedStuff.style.apply(highlight_greaterthan, threshold=10.0, column=['AmplitudeDiff', 'KibanaDiff'], axis=1)
mergedStuff

,SKU,Count,AmplitudeCount,AmplitudeDiff,KibanaCount,KibanaDiff
0,com.pancerdog.tacticool.tier.32,506,503,-1.000000,486,-4.000000
1,com.panzerdog.tacticool.cbp.10,1115,1109,-1.000000,1070,-4.000000
2,com.panzerdog.tacticool.cbp.15,1773,1756,-1.000000,1727,-3.000000
3,com.panzerdog.tacticool.cbp.5,5520,5478,-1.000000,5371,-3.000000
4,com.panzerdog.tacticool.clubpass,8512,3030,-64.000000,2939,-65.000000
5,com.panzerdog.tacticool.hero.common.acc,288,285,-1.000000,276,-4.000000
6,com.panzerdog.tacticool.hero.common.ammo,1259,1240,-2.000000,1192,-5.000000
7,com.panzerdog.tacticool.hero.rare,268,261,-3.000000,253,-6.000000
8,com.panzerdog.tacticool.hero.uncommon.accuracy,57,55,-4.000000,55,-4.000000
9,com.panzerdog.tacticool.hero.uncommon.ammo,58,54,-7.000000,52,-10.000000


In [15]:
mergedStuff.to_excel("Purchase IPhone.xlsx") 

Revenue
----
Для гугла берем как сумму, по AppStore отдельная выгрузка

In [52]:
google_mail = G2006[G2006['Transaction Type'] == 'Charge']
google_mail = google_mail.groupby(['Sku Id']).sum()
google_mail = google_mail[['Amount (Merchant Currency)']]
google_mail.columns = ['Google Revenue']
google_mail.head(5)
google_mail.to_excel("Revenue google.xlsx") 